In [1]:
!pip install flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 70.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 25.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 93.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 32.4 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-

In [2]:
import pandas as pd
from flair.nn import Classifier
from flair.data import Sentence

# load the model
tagger = Classifier.load('sentiment')

2024-02-06 19:30:05,211 https://nlp.informatik.hu-berlin.de/resources/models/sentiment-curated-distilbert/sentiment-en-mix-distillbert_4.pt not found in cache, downloading to /tmp/tmp3mzntrb7


100%|██████████| 253M/253M [00:18<00:00, 14.4MB/s]

2024-02-06 19:30:24,347 copying /tmp/tmp3mzntrb7 to cache at /root/.flair/models/sentiment-en-mix-distillbert_4.pt


2024-02-06 19:30:24,864 removing temp file /tmp/tmp3mzntrb7


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [18]:
data = pd.read_csv("/content/drive/MyDrive/8Semester/preprocessed_reviews.csv", index_col=[0])
nomic_preds = pd.read_csv("/content/drive/MyDrive/8Semester/predictions_nomic_smote.csv", index_col=[0])

In [4]:
data.head()

,product,source,categories,date,didPurchase,doRecommend,rating,reviews,title,review_word_count,year,month,day,hypothesis_label
0,electronics brand product name Tablet A 10.1 T...,Target,"Electronics,iPad & Tablets,All Tablets,XYZ bra...",2021-01-13 00:00:00+00:00,0.0,1.0,5.0,this product so far has not disappointed my ch...,brand name,27,2021.0,1.0,13.0,positive
1,electronics brand product name Tablet A 10.1 T...,Target,"Electronics,iPad & Tablets,All Tablets,XYZ bra...",2021-01-13 00:00:00+00:00,0.0,1.0,5.0,great for beginner or experienced person bough...,very fast,14,2021.0,1.0,13.0,positive
2,electronics brand product name Tablet A 10.1 T...,Target,"Electronics,iPad & Tablets,All Tablets,XYZ bra...",2021-01-13 00:00:00+00:00,0.0,1.0,5.0,inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son.,27,2021.0,1.0,13.0,positive
3,electronics brand product name Tablet A 10.1 T...,Target,"Electronics,iPad & Tablets,All Tablets,XYZ bra...",2021-01-13 00:00:00+00:00,0.0,1.0,4.0,ive had my xyz brand hd num two weeks now and ...,Good!!!,119,2021.0,1.0,13.0,positive
4,electronics brand product name Tablet A 10.1 T...,Target,"Electronics,iPad & Tablets,All Tablets,XYZ bra...",2021-01-12 00:00:00+00:00,0.0,1.0,5.0,i bought this for my grand daughter when she c...,Fantastic Tablet for kids,117,2021.0,1.0,12.0,positive


In [6]:
data = data[data['reviews'].notna()]

In [7]:
sentences = [Sentence(review) for review in data['reviews']]

In [8]:
tagger.predict(sentences=sentences, verbose=True)

Batch inference: 100%|██████████| 1082/1082 [00:45<00:00, 23.61it/s]


In [10]:
predictions = []
for sent in sentences:
  predictions.append(sent.tag.lower())

In [11]:
data['flair_predictions'] = predictions

In [12]:
len(data[data['hypothesis_label']==data['flair_predictions']])/len(data)

0.9116933387255185

In [13]:
neg = data[data['hypothesis_label']=='negative']

In [14]:
len(neg[neg['hypothesis_label']==neg['flair_predictions']])/len(neg)

0.7360754070265638

In [15]:
from sklearn.metrics import precision_recall_fscore_support

In [16]:
precision_recall_fscore_support(data['hypothesis_label'], data['flair_predictions'], average='weighted', labels=['positive', 'negative'])

(0.941568555256199, 0.9116933387255185, 0.9228193163429416, None)

In [17]:
precision_recall_fscore_support(data['hypothesis_label'], data['flair_predictions'], average=None, labels=['positive', 'negative'])

(array([0.97977609, 0.41308007]),
 array([0.92438979, 0.73607541]),
 array([0.95127743, 0.52918528]),
 array([32284,  2334]))

In [19]:
len(data), len(nomic_preds)

(34619, 8665)

In [22]:
nomic_preds = nomic_preds[nomic_preds['review'].notna()]
nomic_preds

,review,label,pred
27712,my home has a lot of wifi product for example ...,positive,positive
29701,i got it because a frend that really lime it i...,negative,positive
22305,the target echo was recommended by our it depa...,positive,positive
34562,got my xyz brandtv box with kodi and is workin...,positive,positive
28364,target echo better known as alexa is much more...,positive,positive
...,...,...,...
6588,i purchased the brand name xyz brand about thr...,positive,positive
20228,bought this brand name for our grandchild and ...,positive,positive
2494,i purchased this xyz brand tablet as a christm...,positive,positive
799,great little tablet for someone who is always ...,positive,positive


In [23]:
nomic_sentences = [Sentence(ns) for ns in nomic_preds['review']]
tagger.predict(nomic_sentences, verbose=True)

Batch inference: 100%|██████████| 271/271 [00:14<00:00, 19.12it/s]


In [24]:
nomic_preds['flair_preds'] = [ns.tag.lower() for ns in nomic_sentences]

<ipython-input-24-9b4200f8fdde>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nomic_preds['flair_preds'] = [ns.tag.lower() for ns in nomic_sentences]


In [25]:
precision_recall_fscore_support(nomic_preds['label'], nomic_preds['flair_preds'], average=None, labels=['positive', 'negative'])

(array([0.97653075, 0.39285714]),
 array([0.92212455, 0.69453925]),
 array([0.94854814, 0.50184957]),
 array([8077,  586]))

In [28]:
pd.DataFrame(
    precision_recall_fscore_support(
        nomic_preds['label'], nomic_preds['flair_preds'], average=None, labels=['positive', 'negative']
        ),
    columns=['Positive', 'Negative'], index=['Precision', 'Recall', 'F1-Score', 'Support'])

,Positive,Negative
Precision,0.976531,0.392857
Recall,0.922125,0.694539
F1-Score,0.948548,0.501850
Support,8077.000000,586.000000


In [29]:
pd.DataFrame(
    precision_recall_fscore_support(
        nomic_preds['label'], nomic_preds['pred'], average=None, labels=['positive', 'negative']
        ),
    columns=['Positive', 'Negative'], index=['Precision', 'Recall', 'F1-Score', 'Support'])

,Positive,Negative
Precision,0.981723,0.368249
Recall,0.904420,0.767918
F1-Score,0.941487,0.497788
Support,8077.000000,586.000000


In [31]:
precision_recall_fscore_support(nomic_preds['label'], nomic_preds['pred'], average='weighted', labels=['positive', 'negative'])

(0.9402250417446819, 0.8951864250259726, 0.9114736814999085, None)

In [32]:
precision_recall_fscore_support(nomic_preds['label'], nomic_preds['flair_preds'], average='weighted', labels=['positive', 'negative'])

(0.9370487269340306, 0.9067297702874293, 0.9183316609416593, None)

In [30]:
nomic_preds.to_csv("/content/drive/MyDrive/8Semester/predictions_nomic_smote.csv")